In [1]:
import pytest

import qcodes.instrument.sims as sims
from qcodes.instrument_drivers.yokogawa.GS200Graph import GS200
from qcodes.graph.visulization import draw
from qcodes.graph.graph import NodeStatus
import networkx as nx
import itertools
VISALIB = sims.__file__.replace("__init__.py", "Yokogawa_GS200.yaml@sim")

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\jenielse\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\jenielse\.qcodes\logs\220505-13244-qcodes.log


# Description

The Yokogawa GS200 is a voltage or current source with an optional measurement unit that may or may not be available depending on the configuration of the instrument.
The measurement unit enables you to measure current when sourcing voltage and the other way around. It is worth noting that the instruments VISA api is highly stateful such that
there is one set command (`:SOUR:LEV`) which will set the voltage or the current depending on the mode set on the instrument (via `:SOUR:FUNC`) 

In qcodes this has previously been implemented as as 2 parameters for voltage and current which both explicitly checks that the instrument is in the correct mode before allowing users to set/get the parameter.
For convenience these have then been delegated to a generic parameter which has its source and unit switched when changing mode on the instrument. 

To ensure correct snapshooting the snapshot status of the voltage/current parameters was enabled/disabled as the mode of the instrument was switched. This was enabled using custom callbacks on mode switching.

In the following we will explore modeling this using a InstrumentModule (Channel) for voltage sourcing and on for current sourcing. These modules may they (depending on if the feature is installed) have a submodule for measuring. 
The idea is that the connection to the device under measurement will be at the main node of the instrument which maps to the Instrument class. When the instrument is in the voltage mode the voltage set (and if exists current measure sub) node will be forwarded 
to the device and the user will not see the other nodes. 


In [2]:
gs200 = GS200("GS200", address="GPIB0::1::INSTR", visalib=VISALIB)

Connected to: QCoDeS Yokogawa Mock GS200 (serial:xxxxxxxx, firmware:MOCKFW) in 0.08s


Lets first draw the graph of the instrument in voltage mode

In [3]:
a = gs200.instrument_graph

adding node GS200
adding node GS200_program
adding node GS200_current_source
adding node GS200_current_source_measure
adding node GS200_voltage_source
adding node GS200_voltage_source_measure


In [4]:
gs200.source_mode("VOLT")

In [5]:
draw(gs200.instrument_graph)

adding node GS200
adding node GS200_program
adding node GS200_current_source
adding node GS200_current_source_measure
adding node GS200_voltage_source
adding node GS200_voltage_source_measure
adding CustomNode(data={'id': 'GS200_container'}, position={})
adding CustomNode(data={'id': 'GS200_container'}, position={})
adding CustomNode(data={'id': 'GS200_container'}, position={})
adding CustomNode(data={'id': 'GS200_container'}, position={})
adding CustomNode(data={'id': 'GS200_container'}, position={})
adding CustomNode(data={'id': 'GS200_container'}, position={})


In [6]:
draw(gs200.instrument_graph.active_subgraph())

adding node GS200
adding node GS200_program
adding node GS200_current_source
adding node GS200_current_source_measure
adding node GS200_voltage_source
adding node GS200_voltage_source_measure
adding CustomNode(data={'id': 'GS200_container'}, position={})
adding CustomNode(data={'id': 'GS200_container'}, position={})
adding CustomNode(data={'id': 'GS200_container'}, position={})
adding CustomNode(data={'id': 'GS200_container'}, position={})
adding CustomNode(data={'id': 'GS200_container'}, position={})


Here we see that the voltage source and the edge from main instrument to voltage source are marked active (Green and blue respectively)

In [7]:
a[('GS200', 'GS200_current_source')].activator.status

<EdgeStatus.INACTIVE_ELECTRICAL_CONNECTION: 'inactive_electrical_connection'>

In [8]:
a[('GS200', 'GS200_voltage_source')].activator.status

<EdgeStatus.ACTIVE_ELECTRICAL_CONNECTION: 'active_electrical_connection'>

In [9]:
a[('GS200')].activator.status

<NodeStatus.ACTIVE: 'active'>

In [10]:
a[('GS200_voltage_source')].activator.status

<NodeStatus.ACTIVE: 'active'>

In [11]:
a[('GS200_current_source')].activator.status

<NodeStatus.INACTIVE: 'inactive'>

And we can see that the status of the repective edges/nodes matches this.

Switching to current mode we can see that things have switched

In [12]:
gs200.source_mode("CURR")

In [13]:
a[('GS200', 'GS200_current_source')].activator.status

<EdgeStatus.ACTIVE_ELECTRICAL_CONNECTION: 'active_electrical_connection'>

In [14]:
a[('GS200', 'GS200_voltage_source')].activator.status

<EdgeStatus.INACTIVE_ELECTRICAL_CONNECTION: 'inactive_electrical_connection'>

In [15]:
a[('GS200')].activator.status

<NodeStatus.ACTIVE: 'active'>

In [16]:
a[('GS200_current_source')].activator.status

<NodeStatus.ACTIVE: 'active'>

In [17]:
a[('GS200', 'GS200_voltage_source')].activator.status

<EdgeStatus.INACTIVE_ELECTRICAL_CONNECTION: 'inactive_electrical_connection'>

In [18]:
a[('GS200_voltage_source')].activator.status

<NodeStatus.INACTIVE: 'inactive'>

In [19]:
draw(gs200.instrument_graph)

adding node GS200
adding node GS200_program
adding node GS200_current_source
adding node GS200_current_source_measure
adding node GS200_voltage_source
adding node GS200_voltage_source_measure
adding CustomNode(data={'id': 'GS200_container'}, position={})
adding CustomNode(data={'id': 'GS200_container'}, position={})
adding CustomNode(data={'id': 'GS200_container'}, position={})
adding CustomNode(data={'id': 'GS200_container'}, position={})
adding CustomNode(data={'id': 'GS200_container'}, position={})
adding CustomNode(data={'id': 'GS200_container'}, position={})


In [20]:
gs200.instrument_graph['GS200']

adding node GS200
adding node GS200_program
adding node GS200_current_source
adding node GS200_current_source_measure
adding node GS200_voltage_source
adding node GS200_voltage_source_measure


<GS200: GS200>

In [21]:
mygraph = gs200.instrument_graph._graph

adding node GS200
adding node GS200_program
adding node GS200_current_source
adding node GS200_current_source_measure
adding node GS200_voltage_source
adding node GS200_voltage_source_measure


In [22]:
def filter_node(nodeid: str) -> bool:
    return mygraph.nodes[nodeid]["value"].activator.status == NodeStatus.ACTIVE

In [23]:
a = nx.subgraph_view(gs200.instrument_graph._graph, filter_node=filter_node)

adding node GS200
adding node GS200_program
adding node GS200_current_source
adding node GS200_current_source_measure
adding node GS200_voltage_source
adding node GS200_voltage_source_measure


In [24]:
node_id = "GS200"

In [25]:
edges = nx.breadth_first_search.bfs_edges(a, node_id)

In [26]:
list(itertools.chain([node_id], map(lambda edge: edge[1], edges)))

['GS200',
 'GS200_program',
 'GS200_current_source',
 'GS200_current_source_measure']

In [28]:
list(gs200.activator.upstream_nodes(gs200.instrument_graph))

adding node GS200
adding node GS200_program
adding node GS200_current_source
adding node GS200_current_source_measure
adding node GS200_voltage_source
adding node GS200_voltage_source_measure


['GS200',
 'GS200_program',
 'GS200_current_source',
 'GS200_current_source_measure']

## Implementation

### General implementation
A node is a class that implements the following interface. This means that an InsturmentModule in it self is a node
```python
class Node(Protocol):

    parameters: Dict[str, _BaseParameter]
    instrument_modules: Dict[str, InstrumentModule]

    @property
    def short_name(self) -> str:
        """Short name of the instrument"""
        pass

    @property
    def full_name(self) -> str:
        """Unique name of the Port with elements separated by .
        Equivalent to parent.name + . + self.short_name
        """
        pass

    @property
    def activator(self) -> NodeActivator:
        ...class Node(Protocol):

    parameters: Dict[str, _BaseParameter]
    instrument_modules: Dict[str, InstrumentModule]

    @property
    def short_name(self) -> str:
        """Short name of the instrument"""
        pass

    @property
    def full_name(self) -> str:
        """Unique name of the Port with elements separated by .
        Equivalent to parent.name + . + self.short_name
        """
        pass

    @property
    def activator(self) -> NodeActivator:
        ...

```

Where the Activator implements the following protocol. (This currently contains the logic from QChar nodes) 
With a few changes. Status is an Enum (so it can contain more values than just off and on) It is possible to query the status

```python
class NodeActivator(abc.ABC):
    def __init__(self, *, node: Node):
        self._node = node
        self._status = NodeStatus.INACTIVE

    @property
    @abc.abstractmethod
    def parameters(self) -> Iterable[Parameter]:
        pass

    def add_source(self, source: Node) -> None:
        _LOG.info(f"Adding Source {source.full_name} to Node: {self.node.full_name}")

    def remove_source(self, source: Node) -> None:
        _LOG.info(
            f"Removing Source {source.full_name} from Node: {self.node.full_name}"
        )

    def activate(self) -> None:
        _LOG.info(f"Activating Node: {self.node.full_name}")

    def deactivate(self) -> None:
        _LOG.info(f"Deactivating Node: {self.node.full_name}")

    @property
    def node(self) -> Node:
        return self._node

    @abc.abstractmethod
    def upstream_nodes(self) -> Iterable[Node]:
        # todo naming
        pass

    @abc.abstractmethod
    def connection_attributes(self) -> Dict[str, Dict[NodeId, ConnectionAttributeType]]:
        pass

    @property
    def status(self) -> NodeStatus:
        return self._statusclass NodeActivator(abc.ABC):
    def __init__(self, *, node: Node):
        self._node = node
        self._status = NodeStatus.INACTIVE

    @property
    @abc.abstractmethod
    def parameters(self) -> Iterable[Parameter]:
        pass

    def add_source(self, source: Node) -> None:
        _LOG.info(f"Adding Source {source.full_name} to Node: {self.node.full_name}")

    def remove_source(self, source: Node) -> None:
        _LOG.info(
            f"Removing Source {source.full_name} from Node: {self.node.full_name}"
        )

    def activate(self) -> None:
        _LOG.info(f"Activating Node: {self.node.full_name}")

    def deactivate(self) -> None:
        _LOG.info(f"Deactivating Node: {self.node.full_name}")

    @property
    def node(self) -> Node:
        return self._node

    @abc.abstractmethod
    def upstream_nodes(self) -> Iterable[Node]:
        # todo naming
        pass

    @abc.abstractmethod
    def connection_attributes(self) -> Dict[str, Dict[NodeId, ConnectionAttributeType]]:
        pass

    @property
    def status(self) -> NodeStatus:
        return self._status
```


This means that it is possible to generically implement this interface for all instruments with a node for each instrument module.






### Specific implementation


To implement the necessary logic the GS200 driver has its own implementation of `_make_instrument_graph`
Which importantly uses a activator that has a mode that depends on the state of a given parameter 

```python
    def _make_instrument_graph(self) -> "StationGraph":
        subgraph_primary_node_names = []
        self_graph = MutableStationGraph()
        self_graph[self.full_name] = self
        subgraphs = [self_graph]
        for submodule in self.instrument_modules.values():
            subgraph = submodule._make_graph()
            subgraph_primary_node_names.append(submodule.full_name)
            subgraphs.append(subgraph)

        graph = MutableStationGraph.compose(*subgraphs)

        for name in subgraph_primary_node_names:
            if "current_source" in name:
                activator = SourceEdgeActivator(
                    status_parameter=self.source_mode, active_state="CURR"
                )
            elif "voltage_source" in name:
                activator = SourceEdgeActivator(
                    status_parameter=self.source_mode, active_state="VOLT"
                )
            else:
                activator = BasicEdgeActivator(edge_status=EdgeStatus.PART_OF)

            graph[self.full_name, name] = Edge(activator=activator)

        return graph.as_station_graph()
```

And by using a custom activator on the current and voltage source modules which also reads its status from a parameter.


```python
        self._activator: NodeActivator = SourceModuleActivator(
            node=self,
            parent=self.parent,
            active_state="VOLT",
            inactive_state="CURR",
            status_parameter=self.root_instrument.source_mode,
        )
        
 ```python
class NodeActivator(abc.ABC):
    def __init__(self, *, node: Node):
        self._node = node
        self._status = NodeStatus.INACTIVE

    @property
    @abc.abstractmethod
    def parameters(self) -> Iterable[Parameter]:
        pass

    def add_source(self, source: Node) -> None:
        _LOG.info(f"Adding Source {source.full_name} to Node: {self.node.full_name}")

    def remove_source(self, source: Node) -> None:
        _LOG.info(
            f"Removing Source {source.full_name} from Node: {self.node.full_name}"
        )

    def activate(self) -> None:
        _LOG.info(f"Activating Node: {self.node.full_name}")

    def deactivate(self) -> None:
        _LOG.info(f"Deactivating Node: {self.node.full_name}")

    @property
    def node(self) -> Node:
        return self._node

    @abc.abstractmethod
    def upstream_nodes(self) -> Iterable[Node]:
        # todo naming
        pass

    @abc.abstractmethod
    def connection_attributes(self) -> Dict[str, Dict[NodeId, ConnectionAttributeType]]:
        pass

    @property
    def status(self) -> NodeStatus:
        return self._statusclass NodeActivator(abc.ABC):
    def __init__(self, *, node: Node):
        self._node = node
        self._status = NodeStatus.INACTIVE

    @property
    @abc.abstractmethod
    def parameters(self) -> Iterable[Parameter]:
        pass

    def add_source(self, source: Node) -> None:
        _LOG.info(f"Adding Source {source.full_name} to Node: {self.node.full_name}")

    def remove_source(self, source: Node) -> None:
        _LOG.info(
            f"Removing Source {source.full_name} from Node: {self.node.full_name}"
        )

    def activate(self) -> None:
        _LOG.info(f"Activating Node: {self.node.full_name}")

    def deactivate(self) -> None:
        _LOG.info(f"Deactivating Node: {self.node.full_name}")

    @property
    def node(self) -> Node:
        return self._node

    @abc.abstractmethod
    def upstream_nodes(self) -> Iterable[Node]:
        # todo naming
        pass

    @abc.abstractmethod
    def connection_attributes(self) -> Dict[str, Dict[NodeId, ConnectionAttributeType]]:
        pass

    @property
    def status(self) -> NodeStatus:
        return self._status
```

## Missing / Open Questions at the moment.


1. How do we enable/disable instrument modules when switching modes. 
    1. Remove the module/ Move to a private module and perhaps replace with empty dummy module. This modifies the state of the instrument, makes typechecking less powerful.
       Module is not shapshotted since its not there anymore
    2. Let parameters get/set check that the module they are bound to is not marked as inactivate. This would require extending the parameter to check status of its bound instrument when getting/setting.
       Using this implementation the . 
       
1. It would probably also be beneficial to be able to invalidate the cache of all parameters in a instrument module (since its not clear that switching from current to voltage and back to current mode would leave all current related parameters as they were)

1. How should parameter forwarding work. Currently the interface suggests that you can do list(gs200.activator.upstream_nodes())[0] to get the source / current module depending on mode. This is less than ideal


1. Visualization.
    1. Style is obviously sub optimal
    2. Currently The graph is directly linked to a the status of the relevant mode parameters as shown above. However the graph when drawn is not interactive (need to call draw again)
    


